Como trabalho final do curso.
Desafio. Prever a inadimplência de uma operação de emprestimo.

Este desafio foi pública inicialmente em 2013.

O desafio inicial pode ser visto em 
https://www.kaggle.com/c/loan-default-prediction/data
    

Sobre o desafio

Essa competição solicita que você determine se um empréstimo será inadimplente, bem como a perda incorrida se não ocorrer. 
Diferentemente das abordagens tradicionais baseadas em finanças para esse problema, onde se distingue entre contrapartes boas ou ruins de uma maneira binária, procuramos antecipar e incorporar o padrão e a gravidade das perdas resultantes. Ao fazer isso, estamos construindo uma ponte entre o sistema bancário tradicional, em que buscamos reduzir o consumo de capital econômico, para uma perspectiva de gerenciamento de ativos, na qual otimizamos o risco para o investidor financeiro.

Sobre o dataset

Esses dados correspondem a um conjunto de transações financeiras associadas a indivíduos. 
Os dados foram padronizados, descendentes e anonimizados. 
Você recebe mais de duzentas mil observações e quase 800 recursos. 
Cada observação é independente da anterior.
Para cada observação, foi registrado se um padrão foi acionado.
Em caso de inadimplência, a perda foi medida. 
Essa quantidade está entre 0 e 100. 
Foi normalizada, considerando que o ideal de cada transação no início é 100. 
Por exemplo, uma perda de 60 significa que apenas 40 são reembolsados. 
Se o empréstimo não for inadimplente, a perda será 0. 
Você deverá prever as perdas para cada observação no conjunto de testes.
Os valores dos recursos ausentes foram mantidos como estão, para que as equipes concorrentes possam realmente usar o máximo de dados disponíveis, implementando uma estratégia para preencher as lacunas, se desejado. 
Observe que algumas variáveis podem ser categóricas (por exemplo, f776 e f777).
O patrocinador da competição trabalhou para remover a dimensionalidade do tempo dos dados. No entanto, as observações ainda estão listadas na ordem do antigo para o novo no conjunto de treinamento. 
No conjunto de teste, eles estão em ordem aleatória.



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl

from scipy import stats
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from IPython.core.pylabtools import figsize

sns.set(font_scale = 2)
%matplotlib inline
mpl.pyplot.rcParams['font.size'] = 24
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

Using TensorFlow backend.


In [3]:
dataset_bruto = pd.read_csv("data/train_v2.csv", header=0)
dataset_bruto.head()

/Users/jcmlima/opt/anaconda3/envs/bernardo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (135,204,274,417) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,...,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0,126.75,126.03,7,0.7607,0.7542,612922,0.7236,0.7236,0.5171,0.7236,0.8476,0.7876,1.097851e+09,89,66,998046.0,89.0,89.0,89.00,89.00,89.0,...,2.3451,0.030594,1.7418,1.5271,0.8474,0.4715,0.028362,3.1611,2.5162,2.0037,0.019636,4.4352,4.2676,-0.1524,1,-0.40,-0.560,-0.440,-0.6280,-3.14,5,2.14,-1.54,1.18,0.1833,0.7873,1,0,5,0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0,123.52,121.35,7,0.6555,0.6555,245815,0.6341,0.6341,0.3909,0.6667,0.6903,0.6903,8.449459e+08,78,50,754416.0,78.0,78.0,78.00,78.00,78.0,...,1.5666,0.120442,1.1963,1.0322,0.4843,0.2389,0.130160,2.7659,1.9523,1.4059,0.115277,3.2763,2.7962,-0.3097,1,-0.17,-0.275,-0.203,-0.2300,-1.38,6,0.54,-0.24,0.13,0.1926,-0.6787,1,0,5,0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,126.49,7,0.7542,0.7542,1385872,0.7542,0.7542,0.5508,0.7542,0.8091,0.7807,1.308478e+09,89,54,1037651.0,89.0,89.0,100.43,94.37,89.0,...,4.5627,0.226336,3.3277,3.4166,1.8321,0.9979,0.103307,6.8623,5.2963,4.1282,0.219729,8.1381,7.3269,-0.1909,1,-0.58,-0.540,-0.572,-0.3985,-5.18,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0,132.94,133.58,7,0.8017,0.7881,704687,0.7881,0.7881,0.5923,0.7881,0.8230,0.8158,1.472752e+09,93,55,1115721.0,93.0,93.0,114.63,102.92,93.0,...,1.6899,0.054630,1.3748,1.3421,0.7982,0.4810,0.081205,2.5571,2.0593,1.6653,0.056470,3.2516,3.0631,-0.1770,1,-0.75,-0.635,-0.745,-0.5100,-2.04,4,1.29,-0.89,0.66,0.2498,0.7119,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,112.77,6,0.5263,0.5263,51985,0.5263,0.5263,0.3044,0.5405,0.5556,0.5455,1.442916e+09,60,21,536400.0,60.0,60.0,60.00,60.00,60.0,...,11.9179,0.085330,7.2175,6.2262,3.1446,1.6149,0.074286,15.9080,12.5688,9.9844,0.067540,17.5561,15.6079,-0.4444,1,-0.18,-0.280,-0.182,-0.4277,-11.12,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0


In [4]:
dataset_bruto.shape

(105471, 771)

In [5]:
dataset_bruto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 771 entries, id to loss
dtypes: float64(653), int64(99), object(19)
memory usage: 620.4+ MB


# Iniciando com uma abordagem simples, 
- Sem exploração dos dados
- em qualquer tratamento
- Com uma rede neural simples.

Dado que a variável a ser prevista é um valor entre 0 e 100, podemos transformar 1 coluna de números inteiros em n colunas booleanas, para cada valor, ou seja, trataremos cada valor como uma categoria.

In [6]:
dataset_numeric = dataset_bruto.apply(pd.to_numeric, errors='coerce')
dataset_numeric.shape

(105471, 771)

In [7]:
dataset_values = dataset_numeric.values 

features = dataset_values[:,1:770].astype(float)
targets = dataset_values[:,770]

print(features.shape)
print(targets.shape)


(105471, 769)
(105471,)


Dado que a variável a ser prevista é um valor entre 0 e 100, podemos transformar 1 coluna de números inteiros em n colunas booleanas, para cada valor, ou seja, trataremos cada valor como uma categoria.

In [8]:
encoder = LabelEncoder()
encoder.fit(targets)

encoded_targets = encoder.transform(targets)
dummy_targets = np_utils.to_categorical(encoded_targets)
dummy_targets.shape

dummy_targets


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Modelo de rede neural "naive". Apenas 1 layer. Como podemos ver como resultado anterior, temos 89 categorias.

In [9]:
model_n = Sequential()
model_n.add(Dense(10, input_dim=769, activation='relu')) # 769 features de entrada
model_n.add(Dense(89, activation='softmax')) # 89 possíveis categorias de saida.
model_n.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_n.summary()
model_n.output_shape

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7700      
_________________________________________________________________
dense_2 (Dense)              (None, 89)                979       
Total params: 8,679
Trainable params: 8,679
Non-trainable params: 0
_________________________________________________________________


(None, 89)

Construido os datasets de treino e de teste

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, dummy_targets, test_size = 0.2, random_state = 42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(84376, 769)
(21095, 769)
(84376, 89)
(21095, 89)


Treino do modelo

In [12]:
history = model_n.fit(X_train, y_train,
                    batch_size=500,
                    epochs=4,
                    verbose=1,
                    validation_data=(X_test, y_test))

Train on 84376 samples, validate on 21095 samples
Epoch 1/4
84376/84376 [==============================] - 5s 56us/step - loss: nan - accuracy: 0.9065 - val_loss: nan - val_accuracy: 0.9060
Epoch 2/4
84376/84376 [==============================] - 4s 43us/step - loss: nan - accuracy: 0.9076 - val_loss: nan - val_accuracy: 0.9060
Epoch 3/4
84376/84376 [==============================] - 4s 43us/step - loss: nan - accuracy: 0.9076 - val_loss: nan - val_accuracy: 0.9060
Epoch 4/4
84376/84376 [==============================] - 4s 43us/step - loss: nan - accuracy: 0.9076 - val_loss: nan - val_accuracy: 0.9060


In [14]:
score = model_n.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0]) # dúvida, por que loss = nan?
print('Test accuracy:', score[1])

Test loss: nan
Test accuracy: 0.9059966802597046


Conclusão, sem grandes esforços obtivemos uma acuraciadade de 90,5%

# Novo modelo
Neste novo modelo, temos 3 camadas, com maior numero de neuronios cada

In [23]:
dataset = dataset_bruto.apply(pd.to_numeric, errors='coerce')
dataset = dataset.values
X = dataset[:,1:770].astype(float)
Y = dataset[:,770]

In [33]:
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Ao invés de manualmete dividirmos as amostrar, podemos utilizar-nos estimators e kfolders, e executarmos o treinamento mais de uma vez.
Esta abordagem, realizamos o treino 3 vezes, cada 1 com 10 épocas e batch_size de 1000

In [34]:
print(features.shape)
print(dummy_targets.shape)
estimator = KerasClassifier(build_fn = model, epochs=50, batch_size=50, verbose=1)
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(estimator, features, dummy_targets, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

(105471, 769)
(105471, 89)
[[126.          10.           0.6868419  ...   1.           0.
    5.        ]
 [121.          10.           0.78277593 ...   1.           0.
    5.        ]
 [126.          10.           0.50007998 ...   1.           0.
    5.        ]
 ...
 [129.           9.           0.24185807 ...   1.           0.
   93.        ]
 [129.           9.           0.56971859 ...   1.           0.
   93.        ]
 [129.           9.           0.40770742 ...   1.           0.
   93.        ]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
Baseline: nan% (nan%)


In [ ]:
#See the column data types and non-missing values
dataset_origin.select_dtypes(include=['object']).head()

In [ ]:
dataset.describe()

In [ ]:
dataset = dataset_origin.apply(pd.to_numeric, errors='coerce')


Análise exploratória do dataset

In [ ]:
dataset.shape


In [ ]:
dataset.dtypes.all

In [ ]:
val = dataset.values
features = val[:,1:770] # a primeira coluna "id" foi ignorada, assim como a ultima coluna, que contém o valor a ser previsto.
loss = val[:,770]  # última coluna "loss" com os valores previstos.

Análise exploratória dos valores de "loss"

In [ ]:
stats.describe(loss)

Os valores de loss estão entre 0 e 100 e são número inteiros.
O aproach foi transformar esta coluna "loss" em 100 variáveis "one hot encoded"

In [ ]:
# o valores a ser previsto 
encoder = LabelEncoder()
encoder.fit(loss)
encoded_loss = encoder.transform(loss)
# conveter mumeros inteiros para categorias≈
dummy_loss = np_utils.to_categorical(encoded_loss)
dummy_loss.shape

O resultado acima mostra que no dataset só existem 89 valores possíveis para loss, assim temos 89 previsões possíveis.

Modelo inicial, para referência,

In [ ]:
model1 = Sequential()
model1.add(Dense(10, input_dim=769, activation='relu')) # 769 features de entrada
model1.add(Dense(89, activation='softmax')) # 89 possíveis categorias de saida.
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()
model1.output_shape


Divisão do dataset entre treino do modelo e teste do modelo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, dummy_loss, test_size=0.20, random_state=42)

Treino do modelo

In [ ]:
history = model1.fit(X_train, y_train,
                    batch_size=500,
                    epochs=3,
                    verbose=1,
                    validation_data=(X_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
score = model1.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0]) # dúvida, por que loss = nan?
print('Test accuracy:', score[1])

Segundo enfoque.
Neste enfoque, utilizaremos uma rede densa com 3 camadas, com 100 neuronios cada

In [ ]:
model2 = Sequential()
model2.add(Dense(100, input_dim=769, activation='relu')) # 769 features de entrada
model2.add(Dense(89, activation='softmax')) # 89 possíveis categorias de saida.
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()
model2.output_shape

Ao invés de dividir a amnostra em duas, iremos utilizar o enfoque kfold, gerando n_splits randomicamente a cada cross_validation.

In [ ]:
# encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(loss)
# encoded_Y = encoder.transform(Y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

model = Sequential()
model.add(Dense(100, input_dim=769, activation='relu'))
model.add(Dense(100, input_dim=769, activation='relu'))
model.add(Dense(89, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.shape

# estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=500, verbose=1)
# kfold = KFold(n_splits=3, shuffle=True)
# results = cross_val_score(estimator, features, dummy_y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))




Para efeitos de comparação, podemos utilizar uma regressão logísitica.

In [ ]:
data_regression = dataset_origin.dropna(axis=1)
data_regression.shape

In [ ]:
values_regression = data_regresion.values
features = values_regression[:,1:245] # a primeira coluna "id" foi ignorada, assim como a ultima coluna, que contém o valor a ser previsto.
loss = values_regression[245]  # última coluna "loss" com os valores previstos.

# encoder = LabelEncoder()
# encoder.fit(loss)
# encoded_loss = encoder.transform(loss)

# dummy_loss = np_utils.to_categorical(encoded_loss)
# dummy_loss.shape




In [ ]:
X, z, Y,  y = train_test_split(features, loss, test_size=0.20, random_state=42)


In [ ]:
clf = LogisticRegression(random_state=0).fit(X,Y)
